# SoftMax Implemention details

[torch 里使用默认使用logsoftmax](https://pytorch.org/docs/stable/_modules/torch/nn/modules/activation.html#Softmax)

```

note::
        This module doesn't work directly with NLLLoss,
        which expects the Log to be computed between the Softmax and itself.
        Use `LogSoftmax` instead (it's faster and has better numerical properties).
```

## Safe-Softmax

先实现基本的[safe-softmax](https://ogunlao.github.io/2020/04/26/you_dont_really_know_softmax.html)

$$
\begin{equation} 
sm(x_i) = \dfrac{e^{x_i - c}}{\sum_{j=1}^{d} e^{x_j -c}}
\end{equation}
$$

## Safe Softmax implemention

In [1]:
import torch
def SoftMax(logits):
    '''
    logits: [batch_size, dim]
    output: [batch_size, dim]
    '''
    logits_max, _ = logits.max(dim = -1)
    logits = logits - logits_max.unsqueeze(1) 
    logits = logits.exp()
    logits_sum = logits.sum(-1, keepdim = True)
    prob = logits / logits_sum
    return prob.abs()
    
logits = torch.randn(8, 10)
prob = SoftMax(logits)
print('原始数据:\n', logits[0]) 
print('softmax:\n', prob[0]) 
print('prob sum:\n',prob[0,:].sum())
print('logprob:\n', prob[0].log())

原始数据:
 tensor([ 1.9187, -0.7027, -1.4287,  0.3837, -0.5099,  0.9975, -0.5943, -0.9596,
         1.3548,  0.5438])

softmax:
 tensor([0.3612, 0.0263, 0.0127, 0.0778, 0.0318, 0.1438, 0.0293, 0.0203, 0.2055,
        0.0913])

prob sum:
 tensor(1.)

logprob:
 tensor([-1.0183, -3.6397, -4.3658, -2.5533, -3.4470, -1.9396, -3.5314, -3.8966,
        -1.5822, -2.3932])

# LogSoftmax

## Softmax overflow

在MLE(最大似然估计中)， 通常要计算logprob(), 以下例子产生了数值溢出: -inf

In [2]:
# 这个例子取log后，会产生数值不稳定性
logits = torch.tensor([[10, 2, 10000, 4]])
prob = SoftMax(logits)
print(logits)
print(prob)
print(prob.log())

tensor([[   10,     2, 10000,     4]])

tensor([[0., 0., 1., 0.]])

tensor([[-inf, -inf, 0., -inf]])

## LogSoftmax Implemention

在pytorch的实现中，使用logsoftmax代替softmax，避免计算logprob产生溢出

可以在原始的Softmax上加入log, 可以推导出logsoftmax

$$
\begin{align}
\text{softmax}(x_i) &= \dfrac{e^{x_i - c}}{\sum_{j=1}^{d} e^{x_j -c}} \\
\text{log\_softmax}(x_i) &= \log \dfrac{e^{x_i - c}}{\sum_{j=1}^{d} e^{x_j -c}} \\
\text{log\_softmax}(x_i) &= x_i - c - \log {\sum_{j=1}^{d} e^{x_j -c}} \\
\end{align} 
$$

此时的logsoftmax得出的logprob不会溢出，同样可以将logprob转化成prob

$$
\text{softmax}(x_i) = \dfrac{e^{\log~probs}}{\sum_{j=1}^{d} e^{\log~probs}}
$$

In [3]:
import torch
def LogSoftMax(logits, recover_prob = True):
    '''
    logits: [batch_size, dim]
    output: [batch_size, dim]
    '''
    # raw_logits = logits
    logits_max, _ = logits.max(dim = -1)
    safe_logits = logits - logits_max.unsqueeze(1)
    safe_logits_exp = safe_logits.exp()
    safe_logits_sum = safe_logits_exp.sum(-1, keepdim = True)
    log_logits_sum = safe_logits_sum.log()
    log_probs = logits - logits_max.unsqueeze(1) - log_logits_sum

    if recover_prob is True:
        exp_log_probs = log_probs.exp()
        sum_log_probs = exp_log_probs.sum(-1, keepdim = True)
        probs = exp_log_probs / sum_log_probs
    
    return probs, log_probs 
    
logits = torch.randn(2, 5)

# softmax
print('--------------softmax------------------')
softmax_probs = SoftMax(logits)
softmax_log_probs = softmax_probs.log()
print(f'softmax probs: \n{softmax_probs}')
print(f'softmax_log_probs: \n{softmax_log_probs}')

# log softmax
print('--------------log softmax------------------')
lsoftmax_probs, lsoftmax_log_probs = LogSoftMax(logits, recover_prob = True)
print(f'logsoftmax probs: \n{lsoftmax_probs}')
print(f'logsoftmax_log_probs: \n{lsoftmax_log_probs}')

--------------softmax------------------

softmax probs: 
tensor([[0.1917, 0.1129, 0.1068, 0.1768, 0.4119],
        [0.4423, 0.1075, 0.3130, 0.0445, 0.0928]])

softmax_log_probs: 
tensor([[-1.6520, -2.1815, -2.2365, -1.7328, -0.8871],
        [-0.8158, -2.2306, -1.1616, -3.1126, -2.3776]])

--------------log softmax------------------

logsoftmax probs: 
tensor([[0.1917, 0.1129, 0.1068, 0.1768, 0.4119],
        [0.4423, 0.1075, 0.3130, 0.0445, 0.0928]])

logsoftmax_log_probs: 
tensor([[-1.6520, -2.1815, -2.2365, -1.7328, -0.8871],
        [-0.8158, -2.2306, -1.1616, -3.1126, -2.3776]])

## Softmax VS LogSoftmax

In [4]:
logits = torch.tensor([[10, 2, 10000, 4]])

# softmax
softmax_probs = SoftMax(logits)
softmax_log_probs = softmax_probs.log()
print('--------------softmax------------------')
print(f'softmax probs: \n{softmax_probs}')
print(f'softmax_log_probs: \n{softmax_log_probs}')

# log softmax
lsoftmax_probs, lsoftmax_log_probs = LogSoftMax(logits, recover_prob = True)
print('--------------log softmax------------------')
print(f'logsoftmax probs: \n{lsoftmax_probs}')
print(f'logsoftmax_log_probs: \n{lsoftmax_log_probs}') 

--------------softmax------------------

softmax probs: 
tensor([[0., 0., 1., 0.]])

softmax_log_probs: 
tensor([[-inf, -inf, 0., -inf]])

--------------log softmax------------------

logsoftmax probs: 
tensor([[0., 0., 1., 0.]])

logsoftmax_log_probs: 
tensor([[-9990., -9998.,     0., -9996.]])

使用`LogSoftmax()` 得到的`logprob` 不会产生`-inf`

# Pytorch LogSoftmax

使用pytorch测试softmax是否会溢出

In [1]:
import torch

x = torch.tensor([[10, 2, 10000, 4]], dtype=torch.float32)

x_softmax = torch.nn.functional.softmax(x)


x_logsoftmax = torch.nn.functional.log_softmax(x)

print(x_softmax)

print(x_softmax.log())

print(x_logsoftmax)

/var/folders/nb/5w4m7xtn6vs7ncl5gs5fs9600000gn/T/ipykernel_17534/742943935.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x_softmax = torch.nn.functional.softmax(x)
/var/folders/nb/5w4m7xtn6vs7ncl5gs5fs9600000gn/T/ipykernel_17534/742943935.py:8: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x_logsoftmax = torch.nn.functional.log_softmax(x)


tensor([[0., 0., 1., 0.]])

tensor([[-inf, -inf, 0., -inf]])

tensor([[-9990., -9998.,     0., -9996.]])